<a href="https://colab.research.google.com/github/danielapavas/ProgramacionLineal/blob/main/Simplex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DANIELA ANDREA PAVAS BEDOYA
# YULY YESENIA ALVEAR ROMO

In [ ]:
pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 66.0 MB/s eta 0:00:00


In [ ]:
import pulp
import re
import pandas as pd

In [ ]:
#Solicitar el número de variables
num_variables = int(input("Ingrese el número de variables: "))

Ingrese el número de variables: 2


In [ ]:
# Crear variables de decisión
variables = [pulp.LpVariable(f'x{i}', lowBound=0) for i in range(num_variables)]

In [ ]:
#  Solicitar  la función objetivo
print("Ingrese la función objetivo en términos de las variables. Por ejemplo, '2*x0 + 3*x1':")
objetivo_str = input("Función objetivo: ")
terms = re.split(r'\s*\+\s*', objetivo_str)
objetivo = pulp.LpAffineExpression()
for term in terms:
    coef, var = term.split('*')
    coef = float(coef)
    if any(v.name == var for v in variables):
        variable = next(v for v in variables if v.name == var)
        objetivo += coef * variable
    else:
        print(f"La variable '{var}' no coincide con las variables definidas.")

# Imprimir la expresión lineal objetivo
print("Función objetivo:", objetivo)


Ingrese la función objetivo en términos de las variables. Por ejemplo, '2*x0 + 3*x1':
Función objetivo: 3*x0 + 5*x1
Función objetivo: 3.0*x0 + 5.0*x1


In [ ]:
num_restricciones = int(input("Ingrese el número de restricciones: "))

var_dict = {v.name: v for v in variables}

restricciones = []
for i in range(num_restricciones):
    restriccion_str = input(f"Ingrese la restricción {i + 1} en términos de las variables (por ejemplo, 'x0+x1 <= 5'): ")

    restriccion_str = re.sub(r'x\d+', lambda x: f'var_dict["{x.group(0)}"]', restriccion_str)

    # Analizar la restricción
    restriccion_parts = re.split(r'\s+', restriccion_str)

    if len(restriccion_parts) != 3:
        print(f"La restricción '{restriccion_str}' no está en el formato correcto.")
    else:
        var1, op, rhs = restriccion_parts

        if op == "<=":
            sense = pulp.LpConstraintLE
        elif op == ">=":
            sense = pulp.LpConstraintGE
        elif op == "=":
            sense = pulp.LpConstraintEQ
        else:
            print(f"Operador '{op}' no válido en la restricción.")
            continue

        restriccion = pulp.LpConstraint(
            e=eval(restriccion_str),
            sense=sense,
            rhs=float(rhs),
            name=f'Restricción_{i}'
        )

        restricciones.append(restriccion)

Ingrese el número de restricciones: 5
Ingrese la restricción 1 en términos de las variables (por ejemplo, 'x0+x1 <= 5'): x0 <= 4
Ingrese la restricción 2 en términos de las variables (por ejemplo, 'x0+x1 <= 5'): 2*x1 <= 12
Ingrese la restricción 3 en términos de las variables (por ejemplo, 'x0+x1 <= 5'): 3*x0+2*x1 <= 18
Ingrese la restricción 4 en términos de las variables (por ejemplo, 'x0+x1 <= 5'): x0 >= 0
Ingrese la restricción 5 en términos de las variables (por ejemplo, 'x0+x1 <= 5'): x1 >= 0


In [ ]:
problema = pulp.LpProblem("Problema", pulp.LpMaximize)  # O maximizar según su caso
problema.setObjective(objetivo)

for restriccion in restricciones:
    problema += restriccion

In [ ]:
problema.solve()

1

In [ ]:
print("Estado:", pulp.LpStatus[problema.status])
print("Valor óptimo de la función objetivo =", pulp.value(problema.objective))
for var in variables:
    print(f"Valor óptimo de {var.name} =", var.varValue)


Estado: Optimal
Valor óptimo de la función objetivo = 72.0
Valor óptimo de x0 = 4.0
Valor óptimo de x1 = 12.0


In [ ]:
# Guardar valores
valor_objetivo_original = pulp.value(problema.objective)
valor_x0_original = variables[0].varValue
valor_x1_original = variables[1].varValue

In [ ]:
# Analisi de sensibilidad al coeficiente x0 en la función objetivo
coeficiente_x0_original = objetivo[variables[0]]
objetivo[variables[0]] = 4.0

In [ ]:
problema.solve()

1

In [ ]:
valor_objetivo_nuevo = pulp.value(problema.objective)
valor_x0_nuevo = variables[0].varValue
valor_x1_nuevo = variables[1].varValue

In [ ]:
# Comparar valores
cambio_objetivo = valor_objetivo_nuevo - valor_objetivo_original
cambio_x0 = valor_x0_nuevo - valor_x0_original
cambio_x1 = valor_x1_nuevo - valor_x1_original

print(f"Cambio en el valor objetivo: {cambio_objetivo}")
print(f"Cambio en x0: {cambio_x0}")
print(f"Cambio en x1: {cambio_x1}")

Cambio en el valor objetivo: 4.0
Cambio en x0: 0.0
Cambio en x1: 0.0


In [ ]:
# Restaurar
objetivo[variables[0]] = coeficiente_x0_original

In [ ]:
# Guardar nuevamente los valores
valor_objetivo_original = pulp.value(problema.objective)
valor_x0_original = variables[0].varValue
valor_x1_original = variables[1].varValue

In [ ]:
problema.solve()

1

In [ ]:
# Analisis de sensibilidad cambio en la restriccion
valor_objetivo_original = pulp.value(problema.objective)

In [ ]:
restricciones[0].rhs = 5

In [ ]:
problema.solve()

1

In [ ]:
valor_objetivo_nuevo = pulp.value(problema.objective)

In [ ]:
rango_de_sombra = valor_objetivo_nuevo - valor_objetivo_original

In [ ]:
print(f"Rango de sombra de la restricción: {rango_de_sombra}")

Rango de sombra de la restricción: 0.0


In [ ]:
print("Estado:", pulp.LpStatus[problema.status])
print("Valor óptimo de la función objetivo =", pulp.value(problema.objective)/2.0)
for var in variables:
    print(f"Valor óptimo de {var.name} =", var.varValue/2.0)

Estado: Optimal
Valor óptimo de la función objetivo = 36.0
Valor óptimo de x0 = 2.0
Valor óptimo de x1 = 6.0


In [ ]:
# Resturar
coeficiente_x0_original = objetivo[variables[0]]
objetivo[variables[0]] = coeficiente_x0_original

restricciones[0].rhs = 4

In [ ]:
# Analisis de sensibilidad  lado derecho de la primera restricción
rhs_original = restricciones[0].rhs


In [ ]:
restricciones[0].rhs = 5

In [ ]:
problema.solve()

1

In [ ]:
valor_objetivo_nuevo = pulp.value(problema.objective)
valor_x0_nuevo = variables[0].varValue
valor_x1_nuevo = variables[1].varValue

In [ ]:
# Comparar valores
cambio_objetivo = valor_objetivo_nuevo - valor_objetivo_original
cambio_x0 = valor_x0_nuevo - valor_x0_original
cambio_x1 = valor_x1_nuevo - valor_x1_original

print(f"Cambio en el valor objetivo: {cambio_objetivo}")
print(f"Cambio en x0: {cambio_x0}")
print(f"Cambio en x1: {cambio_x1}")

Cambio en el valor objetivo: 0.0
Cambio en x0: 0.0
Cambio en x1: 0.0


In [ ]:
problema.solve()

1

In [ ]:
print("Estado:", pulp.LpStatus[problema.status])
print("Valor óptimo de la función objetivo =", pulp.value(problema.objective))
for var in variables:
    print(f"Valor óptimo de {var.name} =", var.varValue)

Estado: Optimal
Valor óptimo de la función objetivo = 76.0
Valor óptimo de x0 = 4.0
Valor óptimo de x1 = 12.0


In [ ]:
print("\nAnálisis de sensibilidad:")
for var in variables:
    print(f"\nRecurso: {var.name}")
    print(f"Rango en el que puede cambiar el recurso y mantener la solución factible: [{var.lowBound}, {var.upBound}]")
    print(f"Valor actual del recurso: {var.varValue}")


Análisis de sensibilidad:

Recurso: x0
Rango en el que puede cambiar el recurso y mantener la solución factible: [0, None]
Valor actual del recurso: 4.0

Recurso: x1
Rango en el que puede cambiar el recurso y mantener la solución factible: [0, None]
Valor actual del recurso: 12.0


In [ ]:
# Crear un DataFrame para almacenar los resultados
resultados_df = pd.DataFrame(columns=["Variable", "Coeficiente", "RHS", "Cambio en Objetivo"] +
                                     [f"Cambio en {var.name}" for var in variables])

# Función para realizar el análisis de sensibilidad y agregar resultados al DataFrame
def agregar_resultado(variable, coeficiente, rhs, cambio_objetivo, cambios_variables):
    resultados_df.loc[len(resultados_df)] = [variable, coeficiente, rhs, cambio_objetivo] + cambios_variables

# Almacenar expresiones originales de restricciones
expresiones_originales = [restriccion.copy() for restriccion in restricciones]

# Analizar sensibilidad para cada variable y restricción
for var in variables:
    # Guardar valores originales de la variable
    valor_original = var.varValue

    # Analizar sensibilidad al coeficiente de la variable en la función objetivo
    coeficiente_original = objetivo[var]
    objetivo[var] = 4.0
    problema.solve()
    valor_nuevo = var.varValue
    cambio_objetivo = pulp.value(problema.objective) - valor_objetivo_original
    cambios_variables = [var_nuevo - valor_original for var_nuevo, valor_original in zip([var.varValue for var in variables], [valor_original for _ in variables])]
    agregar_resultado(var.name, coeficiente_original, None, cambio_objetivo, cambios_variables)

    # Restaurar valores originales
    objetivo[var] = coeficiente_original

# Analizar sensibilidad para cada restricción
for i, restriccion in enumerate(restricciones):
    # Guardar expresión original de la restricción
    restricciones[i] = expresiones_originales[i]

    # Analizar sensibilidad al lado derecho de la restricción
    restricciones[i].rhs = 5
    problema.solve()
    cambio_objetivo = pulp.value(problema.objective) - valor_objetivo_original
    cambios_variables = [var.varValue - valor_original for var, valor_original in zip(variables, [valor_original for _ in variables])]
    agregar_resultado(f"Restricción_{i + 1}", None, restricciones[i].rhs, cambio_objetivo, cambios_variables)

    # Restaurar expresión original
    restricciones[i] = expresiones_originales[i]

# Mostrar los resultados en forma tabular
print(resultados_df)


        Variable  Coeficiente   RHS  Cambio en Objetivo  Cambio en x0  \
0             x0          4.0  None                 4.0           0.0   
1             x1          5.0  None                -8.0          -8.0   
2  Restricción_1          NaN     5                 4.0          -8.0   
3  Restricción_2          NaN     5                 4.0          -8.0   
4  Restricción_3          NaN     5                 4.0          -8.0   
5  Restricción_4          NaN     5                 4.0          -8.0   
6  Restricción_5          NaN     5                 4.0          -8.0   

   Cambio en x1  
0           8.0  
1           0.0  
2           0.0  
3           0.0  
4           0.0  
5           0.0  
6           0.0  


In [ ]:
print("Estado:", pulp.LpStatus[problema.status])
print("Valor óptimo de la función objetivo =", pulp.value(problema.objective)/2.0)
for var in variables:
    print(f"Valor óptimo de {var.name} =", var.varValue/2.0)

Estado: Optimal
Valor óptimo de la función objetivo = 38.0
Valor óptimo de x0 = 2.0
Valor óptimo de x1 = 6.0


In [ ]:
    resultado_iteracion = {'Iteración': i + 1}
    for var in variables:
        resultado_iteracion[f'Valor actual de {var.name}'] = var.varValue
    resultado_iteracion['Valor óptimo de la función objetivo'] = pulp.value(problema_copia.objective)

    resultados_analisis.append(resultado_iteracion)

In [ ]:
df_resultados = pd.DataFrame(resultados_analisis)
print("\nAnálisis de Sensibilidad en Forma de Tabla:")
print(df_resultados)

print("\nAnálisis de sensibilidad:")
for var in variables:
    print(f"\nRecurso: {var.name}")
    print(f"Rango en el que puede cambiar el recurso y mantener la solución factible: [0, {var.upBound}]")
    print(f"Valor actual del recurso: {var.varValue}")


Análisis de Sensibilidad en Forma de Tabla:
   Iteración  Valor actual de x0  Valor actual de x1  \
0          1                 4.0                12.0   

   Valor óptimo de la función objetivo  
0                                 76.0  

Análisis de sensibilidad:

Recurso: x0
Rango en el que puede cambiar el recurso y mantener la solución factible: [0, None]
Valor actual del recurso: 4.0

Recurso: x1
Rango en el que puede cambiar el recurso y mantener la solución factible: [0, None]
Valor actual del recurso: 12.0


In [ ]:
resultados_analisis = []
num_iteraciones = 10


In [ ]:

for i in range(num_iteraciones):
    variables_copia = [pulp.LpVariable(f'x{i}', lowBound=0) for i in range(num_variables)]

    objetivo_copia = objetivo.copy()
    problema_copia = pulp.LpProblem("Problema_copia", pulp.LpMaximize)
    for restriccion in restricciones:

        problema_copia += restriccion
    problema_copia.setObjective(objetivo_copia)
    problema_copia.solve()


    resultado_iteracion = {'Iteración': i + 1}
    for j, var in enumerate(variables_copia):
        resultado_iteracion[f'Valor actual de {var.name}'] = var.varValue
    resultado_iteracion['Valor óptimo de la función objetivo'] = pulp.value(problema_copia.objective)

    resultados_analisis.append(resultado_iteracion)


df_resultados = pd.DataFrame(resultados_analisis)

# Mostrar la tabla de resultados
print("\nAnálisis de Sensibilidad en Forma de Tabla:")
print(df_resultados)


Análisis de Sensibilidad en Forma de Tabla:
    Iteración Valor actual de x0 Valor actual de x1  \
0           1               None               None   
1           2               None               None   
2           3               None               None   
3           4               None               None   
4           5               None               None   
5           6               None               None   
6           7               None               None   
7           8               None               None   
8           9               None               None   
9          10               None               None   
10          1               None               None   
11          2               None               None   
12          3               None               None   
13          4               None               None   
14          5               None               None   
15          6               None               None   
16          7       

In [ ]:
from pulp import LpProblem, LpVariable, lpSum, LpMaximize
import pandas as pd

prob = LpProblem("Maximizar_Objetivo", LpMaximize)

x0 = LpVariable("x0", lowBound=0)
x1 = LpVariable("x1", lowBound=0)

prob += 3.0 * x0 + 5.0 * x1, "Objetivo"

prob += x0 <= 4, "Restriccion_1"
prob += 2 * x1 <= 12, "Restriccion_2"
prob += 3 * x0 + 2 * x1 <= 18, "Restriccion_3"

# Resolver el problema
prob.solve()

print(f"Estado: {prob.status}, Valor Objetivo: {prob.objective.value()}")

for v in prob.variables():
    print(f"{v.name}: {v.value()}")

# Realizar el análisis de sensibilidad
datos_sensibilidad = []

for nombre, c in prob.constraints.items():
    precio_sombra = c.pi
    excedente = c.slack

    datos_sensibilidad.append({
        "Restriccion": nombre,
        "Precio Sombra": precio_sombra,
        "Excedente": excedente,
    })

# Crear la tabla de sensibilidad utilizando Pandas
tabla_sensibilidad = pd.DataFrame(datos_sensibilidad)
print("\nAnálisis de Sensibilidad:")
print(tabla_sensibilidad)

Estado: 1, Valor Objetivo: 36.0
x0: 2.0
x1: 6.0

Análisis de Sensibilidad:
     Restriccion  Precio Sombra  Excedente
0  Restriccion_1           -0.0        2.0
1  Restriccion_2            1.5       -0.0
2  Restriccion_3            1.0       -0.0


In [ ]:
from pulp import LpProblem, LpVariable, LpMaximize

# Crear el problema de programación lineal
prob = LpProblem("Maximizar_Objetivo", LpMaximize)

x0 = LpVariable("x0", lowBound=0)
x1 = LpVariable("x1", lowBound=0)

prob += 3.0 * x0 + 5.0 * x1, "Objetivo"

prob += x0 <= 4, "Restriccion_1"
prob += 2 * x1 <= 12, "Restriccion_2"
prob += 3 * x0 + 2 * x1 <= 18, "Restriccion_3"

prob.solve()

# Mostrar los resultados iniciales
print(f"Estado: {prob.status}, Valor Objetivo: {prob.objective.value()}")
for v in prob.variables():
    print(f"{v.name}: {v.value()}")

# Realizar el análisis de sensibilidad (cambiar coeficientes de la función objetivo)
datos_sensibilidad = []

for coeficiente in [1.0, 2.0, 3.0, 4.0, 5.0]:
    nueva_func_objetivo = 3.0 * coeficiente * x0 + 5.0 * x1
    prob.objective = nueva_func_objetivo

    prob.solve()
    datos_sensibilidad.append({
        "Coeficiente_x0": coeficiente,
        "Valor_Objetivo": prob.objective.value(),
        "x0": x0.value(),
        "x1": x1.value(),
    })

# Crear la tabla de sensibilidad
tabla_sensibilidad = pd.DataFrame(datos_sensibilidad)
print("\nAnálisis de Sensibilidad Cambiando Coeficiente de x0 en la Función Objetivo:")
print(tabla_sensibilidad)

Estado: 1, Valor Objetivo: 36.0
x0: 2.0
x1: 6.0

Análisis de Sensibilidad Cambiando Coeficiente de x0 en la Función Objetivo:
   Coeficiente_x0  Valor_Objetivo   x0   x1
0             1.0            36.0  2.0  6.0
1             2.0            42.0  2.0  6.0
2             3.0            51.0  4.0  3.0
3             4.0            63.0  4.0  3.0
4             5.0            75.0  4.0  3.0
